In [1]:
import numpy as np 
import scipy
import torch
import torch.nn as nn
import torch.optim as optim

from numpy import genfromtxt

In [17]:
import os
current_directory = os.getcwd()
print(current_directory)

/Users/vikrant/Documents/VT/Classes/deep-learning/Final-project/real_time


In [23]:
file_name_prefix = "/Users/vikrant/Documents/VT/Classes/deep-learning/Final-project/real_time/UCRArchive_2018/Adiac/Adiac"

In [27]:
# Load train and test data

train_data_with_labels = np.genfromtxt(file_name_prefix+'_TRAIN.tsv')
test_data_with_labels = np.genfromtxt(file_name_prefix+'_TEST.tsv')

data_with_labels = np.vstack( (train_data_with_labels, test_data_with_labels))
data = data_with_labels[:,1:]
labels = data_with_labels[:,0]


# We make sure that labels are numbered as 0, 1, 2, ... 
# and set the number of classes

min_label = min(labels)
max_label = max(labels)
if min_label == 0:
  NUM_CLASSES = int(max_label+1)
elif min_label == 1:
  labels = labels - min_label
  NUM_CLASSES = int(max_label)
elif min_label == -1:
  if np.sum(labels == -1)+np.sum(labels==1) == len(labels):
    NUM_CLASSES = 2
    labels[labels==-1]=0
  else:
    raise Exception("Unexpected labels")
else:
  raise Exception("Unexpected labels")

# We make sure that the length of the time series is a multiple of 4 
# (for Net1, we acutally only need the length to be a multiple of 2,
# but we need the length to be a multiple of 4 for Net2)

NUM_INPUT_FEATURES = len(data[0]) 
values_to_cut = NUM_INPUT_FEATURES % 4
if values_to_cut != 0:
  data = data[:,0:NUM_INPUT_FEATURES-values_to_cut]
  NUM_INPUT_FEATURES = NUM_INPUT_FEATURES - values_to_cut

In [33]:
pip install cython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 738.4 kB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [35]:
%load_ext cython

In [37]:
%%cython

import numpy as np
cimport numpy as np

def dtw(np.ndarray[np.float_t, ndim=1] ts1, 
        np.ndarray[np.float_t, ndim=1] ts2):
  
  cdef int LEN_TS1 
  cdef int LEN_TS2
  cdef int i
  cdef int j
  cdef np.ndarray[np.float_t, ndim=2] dtw_matrix

  LEN_TS1 = len(ts1)
  LEN_TS2 = len(ts2)

  dtw_matrix = np.zeros( (LEN_TS1, LEN_TS2), dtype=np.float )
  
  dtw_matrix[0,0] = abs(ts1[0]-ts2[0])
  
  for i in range(1, LEN_TS1):
    dtw_matrix[i,0] = dtw_matrix[i-1,0]+abs(ts1[i]-ts2[0])

  for j in range(1, LEN_TS2):
    dtw_matrix[0,j] = dtw_matrix[0,j-1]+abs(ts1[0]-ts2[j])

  for i in range(1, LEN_TS1):
    for j in range(1, LEN_TS2):
      dtw_matrix[i,j] = min(dtw_matrix[i-1,j-1], dtw_matrix[i-1,j], 
                            dtw_matrix[i, j-1]) + abs(ts1[i]-ts2[j])
  
  return dtw_matrix[ len(ts1)-1, len(ts2)-1 ]

Content of stderr:
In file included from /Users/vikrant/.ipython/cython/_cython_magic_9c771bd9c84b20df6b876af06aa8362c247b22b8.c:1250:
In file included from /opt/anaconda3/lib/python3.12/site-packages/numpy/core/include/numpy/arrayobject.h:5:
In file included from /opt/anaconda3/lib/python3.12/site-packages/numpy/core/include/numpy/ndarrayobject.h:12:
In file included from /opt/anaconda3/lib/python3.12/site-packages/numpy/core/include/numpy/ndarraytypes.h:1929:
/opt/anaconda3/lib/python3.12/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: "Using deprecated NumPy API, disable it with "          "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-W#warnings]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^
1 warning generated.
ld: warning: duplicate -rpath '/opt/anaconda3/lib' ignored

In [39]:
def dc_activations(data, convolutional_filters):
  """
  Calculation of the activations of the dynamic convolutional layer.

  Inputs
  ------
    data : np.array 
      Two-dimensional array containing the input data, 
      each row of the array corresponds to one of the time series
    convolutional_filters : np.array
      Three-dimensional array containing the parameters of the dynamic 
      convolutional layer. The first index corresponds to the output channel
      of the convolution, the second index corresponds to the input channel 
      (the current implementation only works with 1 input channel, so the second
      index is always zero), the third index is the position within the local
      pattern corresponding to a convolutional filter
  """
  num_instances = len(data)
  length_of_time_series = len(data[0])
  num_conv_filters = len(convolutional_filters)
  conv_filter_size = len(convolutional_filters[0][0])

  activations = np.zeros( (num_instances, num_conv_filters, 
                           length_of_time_series-conv_filter_size+1) )
  for i in range(num_instances):
    for j in range(length_of_time_series-conv_filter_size+1):
      for k in range(num_conv_filters):
        activations[i,k,j] = dtw(convolutional_filters[k][0],
                                 data[i,j:j+conv_filter_size])
        
  return activations

In [41]:
CONV_FILTERS = 25
CONV_FILTERS2 = 10
CONV_FILTER_SIZE = 9

class Net1_CNN(nn.Module):
    def __init__(self):
        super(Net1_CNN, self).__init__()
        num_units_fc = 100
        self.num_inputs_fc = int(CONV_FILTERS*(NUM_INPUT_FEATURES-CONV_FILTER_SIZE+1)/2)

        self.conv1 = nn.Conv1d(in_channels = 1, out_channels = CONV_FILTERS, 
                               kernel_size=CONV_FILTER_SIZE, padding = 0, stride = 1)
        self.max_pool = nn.MaxPool1d(2)
        self.fc = nn.Linear(self.num_inputs_fc, num_units_fc)
        self.out = nn.Linear(num_units_fc, NUM_CLASSES) 

    def forward(self, x):
        x = x.view(-1, 1, NUM_INPUT_FEATURES)
        x = self.conv1(x)
        x = self.max_pool(x)
        x = x.view(-1, self.num_inputs_fc)
        x = torch.relu(self.fc(x))
        x = self.out(x)
        return x


class Net2_CNN(nn.Module):
    def __init__(self):
        super(Net2_CNN, self).__init__()
        num_units_fc = 100
        self.num_inputs_fc = int(CONV_FILTERS2*((NUM_INPUT_FEATURES-CONV_FILTER_SIZE+1)/2-CONV_FILTER_SIZE+1)/2)

        self.conv1 = nn.Conv1d(in_channels = 1, out_channels = CONV_FILTERS, 
                               kernel_size=CONV_FILTER_SIZE, padding = 0, stride = 1)
        self.max_pool = nn.MaxPool1d(2)

        self.conv2 = nn.Conv1d(in_channels = CONV_FILTERS, out_channels = CONV_FILTERS2, 
                               kernel_size=CONV_FILTER_SIZE, padding = 0, stride = 1)
        self.max_pool2 = nn.MaxPool1d(2)
        self.fc = nn.Linear(self.num_inputs_fc, num_units_fc)
        self.out = nn.Linear(num_units_fc, NUM_CLASSES) 

    def forward(self, x):
        x = x.view(-1, 1, NUM_INPUT_FEATURES)
        x = self.conv1(x)
        x = self.max_pool(x)
        x = self.conv2(x)
        x = self.max_pool2(x)
        x = x.view(-1, self.num_inputs_fc)
        x = torch.relu(self.fc(x))
        x = self.out(x)
        return x


# Please note that the dynamic convolutional layer is initialized using the 
# parameters learned during the "pre-train" phase (in which a "usual" 
# convolutional network is trained). Once the pre-train phased is completed, 
# the parameters of the dynamic convoltuional layer are fixed, therefore,
# the activations of the dynamic convolutional layer will be pre-calculated 
# outside the network for efficient implementation.

class Net1_DCNN(nn.Module):
    def __init__(self):
        super(Net1_DCNN, self).__init__()
        num_units_fc = 100

        self.max_pool = nn.MaxPool1d(2)
        self.fc = nn.Linear(int(CONV_FILTERS*(NUM_INPUT_FEATURES-CONV_FILTER_SIZE+1)/2), num_units_fc)
        self.out = nn.Linear(num_units_fc, NUM_CLASSES) 

    def forward(self, x):
        x = x.view(-1, CONV_FILTERS, NUM_INPUT_FEATURES-CONV_FILTER_SIZE+1)
        x = self.max_pool(x)
        x = x.view(-1,int(CONV_FILTERS*(NUM_INPUT_FEATURES-CONV_FILTER_SIZE+1)/2))
        x = torch.relu(self.fc(x))
        x = self.out(x)
        return x


class Net2_DCNN(nn.Module):
    def __init__(self):
        super(Net2_DCNN, self).__init__()
        num_units_fc = 100
        self.num_inputs_fc = int(CONV_FILTERS2*((NUM_INPUT_FEATURES-CONV_FILTER_SIZE+1)/2-CONV_FILTER_SIZE+1)/2)

        self.max_pool = nn.MaxPool1d(2)

        self.conv2 = nn.Conv1d(in_channels = CONV_FILTERS, out_channels = CONV_FILTERS2, 
                               kernel_size=CONV_FILTER_SIZE, padding = 0, stride = 1)
        self.max_pool2 = nn.MaxPool1d(2)
        self.fc = nn.Linear(self.num_inputs_fc, num_units_fc)
        self.out = nn.Linear(num_units_fc, NUM_CLASSES) 
 

    def forward(self, x):
        x = x.view(-1, CONV_FILTERS, NUM_INPUT_FEATURES-CONV_FILTER_SIZE+1)
        x = self.max_pool(x)
        x = self.conv2(x)
        x = self.max_pool2(x)
        x = x.view(-1, self.num_inputs_fc)
        x = torch.relu(self.fc(x))
        x = self.out(x)
        return x

In [43]:
def eval_net(net, test_data, device):
  test_dataset = torch.utils.data.TensorDataset( 
    torch.Tensor(test_data), 
    torch.LongTensor(test_labels)
  )
  testloader = torch.utils.data.DataLoader(test_dataset)

  correct = 0
  total = 0

  with torch.no_grad():
    for inputs, targets in testloader:
      inputs = inputs.to(device) 
      targets = targets.to(device) 
      outputs = net(inputs)
      _, predicted = torch.max(outputs.data, 1)

      total += targets.size(0)
      correct += (predicted == targets)
  
  return float(correct/total), float(correct), float(total)

In [48]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

np.float = float 
# If available, we use the GPU to train and evaluate the network
# (If using Google Colab, make sure that you select a runtime with GPU in
# the menu item "Edit/notebook settings" or "Runtime/Change runtime type".)

if torch.cuda.is_available():
  device = torch.device("cuda:0")
  print("Train on GPU")
else:
  device = torch.device("cpu")
  print("GPU is not available, we will use CPU instead of GPU")

# DTW computations will be executed on the CPU
cpu_device = torch.device("cpu")

accuracies_cnn = []
accuracies_dcnn = []
skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

fold = 0
for train_index, test_index in skf.split(data, labels):
  fold = fold + 1

  train_data = data[train_index]
  train_labels = labels[train_index]
  test_data = data[test_index]
  test_labels = labels[test_index]

  # Training of CNN. This is simultaneously the pre-train phase of DCNN.

  train_dataset = torch.utils.data.TensorDataset(
      torch.Tensor(train_data), 
      torch.LongTensor(train_labels) 
  )
  trainloader = torch.utils.data.DataLoader(
      train_dataset, shuffle=True, batch_size=16)

  cnn = Net2_CNN()

  #use:
  #cnn = Net1_CNN() for experiment with Net1 from the manuscript
  #cnn = Net2_CNN() for experiment with Net2 from the manuscript
  
  cnn.to(device)

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(cnn.parameters(), lr=1e-5)

  running_loss = 0.0
  running_n = 0

  print("Training CNN...")

  for epoch in range(1000):  
    for inputs, targets in trainloader:
      inputs = inputs.to(device) 
      targets = targets.to(device) 
      optimizer.zero_grad()
      
      outputs = cnn(inputs)

      loss = criterion(outputs, targets)
      loss.backward()
      optimizer.step()

      running_loss += loss.item()
      running_n = running_n + 1

    if epoch % 100 == 0:
      print("epoch: {:3d} loss: {:4.3f}".format(epoch, running_loss/running_n))
      running_loss = 0.0
      running_n = 0

  # Obtain the parameters of the dynamic convolutional layer, and 
  # precalculate its activations
  params = []
  for p in cnn.parameters():
    params.append(p)

  convolutional_filters = np.array(params[0].to(cpu_device).detach().numpy(), 
                                   dtype=np.float)

  dc_activations_train = dc_activations(train_data, convolutional_filters)

  # Train DCNN

  train_dataset = torch.utils.data.TensorDataset(
      torch.Tensor(dc_activations_train), 
      torch.LongTensor(train_labels) 
  )
  trainloader = torch.utils.data.DataLoader(
      train_dataset, shuffle=True, batch_size=16)


  #use:  
  #dcnn = Net1_DCNN() for experiments with Net1 from the manuscript
  #dcnn = Net2_DCNN() for experiments with Net2 from the manuscript

  dcnn = Net2_DCNN()

  dcnn.to(device)

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(dcnn.parameters(), lr=1e-5)

  running_loss = 0.0
  running_n = 0

  print("Training DCNN...")

  for epoch in range(1000):  
    for inputs, targets in trainloader:
      inputs = inputs.to(device) 
      targets = targets.to(device) 
      optimizer.zero_grad()
      
      outputs = dcnn(inputs)

      loss = criterion(outputs, targets)
      loss.backward()
      optimizer.step()

      running_loss += loss.item()
      running_n = running_n + 1

    if epoch % 100 == 0:
      print("epoch: {:3d} loss: {:4.3f}".format(epoch, running_loss/running_n))
      running_loss = 0.0
      running_n = 0
  
  acc, _, _ = eval_net(cnn, test_data, device)
  accuracies_cnn.append(acc)
  print("Fold: {:2d}, accuracy of CNN:  {:4.3f}".format(fold, acc))

  dc_activations_test = dc_activations(test_data, convolutional_filters)
  acc, _, _ = eval_net(dcnn, dc_activations_test, device)
  print("Fold: {:2d}, accuracy of DCNN: {:4.3f}".format(fold, acc))
  accuracies_dcnn.append(acc)


GPU is not available, we will use CPU instead of GPU
Training CNN...
epoch:   0 loss: 3.614
epoch: 100 loss: 3.449
epoch: 200 loss: 2.878
epoch: 300 loss: 2.427
epoch: 400 loss: 2.143
epoch: 500 loss: 1.949
epoch: 600 loss: 1.799
epoch: 700 loss: 1.674
epoch: 800 loss: 1.563
epoch: 900 loss: 1.461
Training DCNN...
epoch:   0 loss: 3.945
epoch: 100 loss: 3.261
epoch: 200 loss: 2.706
epoch: 300 loss: 2.311
epoch: 400 loss: 2.028
epoch: 500 loss: 1.827
epoch: 600 loss: 1.674
epoch: 700 loss: 1.551
epoch: 800 loss: 1.444
epoch: 900 loss: 1.349
Fold:  1, accuracy of CNN:  0.506
Fold:  1, accuracy of DCNN: 0.620
Training CNN...
epoch:   0 loss: 3.615
epoch: 100 loss: 3.447
epoch: 200 loss: 2.905
epoch: 300 loss: 2.457
epoch: 400 loss: 2.115
epoch: 500 loss: 1.895
epoch: 600 loss: 1.742
epoch: 700 loss: 1.624
epoch: 800 loss: 1.529
epoch: 900 loss: 1.447
Training DCNN...
epoch:   0 loss: 4.073
epoch: 100 loss: 3.228
epoch: 200 loss: 2.487
epoch: 300 loss: 2.021
epoch: 400 loss: 1.743
epoch: 5

In [49]:
print("**{}**\n\n".format(file_name_prefix.split('/')[-1]))
print("\t\tp-value:        {:4.3f}".format(scipy.stats.ttest_rel(accuracies_cnn, accuracies_dcnn)[1]))
print("\t\tMean acc. CNN:  {:4.3f}".format(np.mean(accuracies_cnn)))
print("\t\tMean acc. DCNN: {:4.3f}".format(np.mean(accuracies_dcnn)))
print("\t\tStd. acc. CNN:  {:4.3f}".format(np.std(accuracies_cnn)))
print("\t\tStd. acc. DCNN: {:4.3f}".format(np.std(accuracies_dcnn)))

**Adiac**


		p-value:        0.001
		Mean acc. CNN:  0.571
		Mean acc. DCNN: 0.653
		Std. acc. CNN:  0.059
		Std. acc. DCNN: 0.062
